In [2]:
import os,sys,inspect
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from datetime import datetime
from agents.Base import deepagents, run
from agents.BlueAgents import *
from data.preprocessing import data_split, get_price, DataProcessor
import pandas as pd
import numpy as np
from agents.wrappers.ObservationWrapper import NormalizedEnv
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from env.BaseEnv import EnvConfig
from env.EnvStock_train import StockEnvTrain
from env.EnvStock_trade import StockEnvTrade
from env.EnvStock_val import StockEnvValidation
from env.crypto.SingleCryptoEnv import SingleCryptoEnv
from env.crypto.SingleCryptoEnv import SingleCryptoEnv
from env.BaseEnv import EnvConfig
from agents.BlueAgents import TrainerConfig

%matplotlib inline
%load_ext autoreload
%autoreload 2

C:\Users\kaanb\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\kaanb\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#depre

In [3]:
df = pd.read_csv('../datasets/downew30.csv')
cwd = '../AgentResult'

In [4]:
indata = data_split(df, '2010-01-01','2021-12-25')

In [5]:
# Winner hparams with 3.1m ending for PPO
# environment reward is grade 0
# indicator based
# Error on reward where the agent get -500 

last_winner ={'eval_time_gap': 32, 'batch_size': 32, 
 'gamma': 0.98, 'learning_rate': 0.03372, 
 'target_step': 86175, 'net_dimension': 1026, 'episode': 3, 'seed': 42069}

rec_decay ={'eval_time_gap': 32, 'batch_size': 32, 
 'gamma': 0.98, 'learning_rate': {'start':0.03372, 'end':0.0001}, 
 'target_step': 86175, 'net_dimension': 1026, 'episode': 20, 'seed': 42069}

# Own agent

1. Winner params
'''
    model_kwargs = {
        'learning_rate':0.0027,
        'batch_size':64,
        'gamma':0.99,
        'seed':42069,
        'net_dimension':256, # Change this dimension to be more dynamic
        'target_step':5000,
        'eval_time_gap':10
    }

    model_name = 'jim-ppo-v2'
    model_type = 'ppo'

    with indicators without open high low close and without the episodic training
    
    rebalance_window = 63
    validation_window = 63
'''

- [x] Change the timesteps version make sure it does reiterate with same agent
- [x] Normalize Inputs for the model
- [x] Normalize reward function
- [ ] Vectorized paralel envs
- [x] start trading from 2016
- [x] Add PBT
- [x] Change neet dimension problem here you cannot make it another than 160 
- [x] Create summary write for tensorboard
- [ ] Input dictionary
- [ ] Auxilary Loss
- [x] LR Decay
- [ ] Loss on sell -- loss compared to buy and hold of stock
- [ ] Stock Randomization (Automatic Domain Randomization)
- [ ] Implement shared layer and seperate networks for RNN's
- [x] **CHANGE TRAINGIN PROCESS WHERE IT CREATES TRAJECTORIES WITH THE NEW UPDATED POLCIY**
- [ ] Implement Early stopping
- [ ] Add time-frame to data /check whats added
- [x] Training with giving a stocks information together, not indicator based style
- [ ] Decision Transformer
- [ ] Win / Loss multi agent training
- [ ] kelly criterion as auxilary task / optimize kelly main obj

In [5]:
MODELS = ['ddpg','ppo','recurrent_ppo']

model_kwargs = {
    'learning_rate':0.0035,
    'batch_size':16,
    'gamma':0.99,
    'seed':6931,
    'net_dimension':1024, 
    'target_step':42500,
    'eval_time_gap':11,
    "episode":40 # NONE if you only want to give target step
}

second = {'eval_time_gap': 16, 'batch_size': 16, 'gamma': 0.999, 'learning_rate': 0.04498, 
          'target_step': 80242, 'net_dimension': 128, 'episode': 5, 'seed': 31} # This lost to bnh in crisis
model_name = 'jim-ppo-recurrent'
model_type = MODELS[1]
env_config = EnvConfig(stock_first_state= False, reward_grade=1)
tconfig = TrainerConfig(dataset_version = 'DOW30', 
                        population = 25, normalize_env=True, 
                        rebalance_window=126, validation_window=126)
# For trading dolars in the environment set hmax normalize which is max dollar amount to buy in one order
env_config_dollar_trade = EnvConfig(stock_first_state= False, INITIAL_ACCOUNT_BALANCE= 10000, quantity_buying=False, HMAX_NORMALIZE=500)

In [6]:
agent_builder = DRLAgent(model_name ,model_type, StockEnvTrain, 
                         StockEnvValidation, 
                         StockEnvTrade, indata, last_winner, tconfig, config = env_config)


In [ ]:
agent_builder.run_prediction(total_timesteps=1000)

{'eval_time_gap': 32, 'batch_size': 32, 'gamma': 0.98, 'learning_rate': 0.03372, 'target_step': 86175, 'net_dimension': 1026, 'episode': 3, 'seed': 42069}
======Model training from:  2010-01-01 to  2016-01-04
| Remove cwd: ./trained_models
Using device:  cpu
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]C:\Users\kaanb\Desktop\Code\Finance\Bot\FundSimulator\agents\agents\ppo.py:297: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  ten_reward = torch.as_tensor(temp[1], dtype=torch.float32) * reward_scale


In [7]:
agent_builder.run_pbt_prediction()

======Training Agents with the population of 25========
Agent:  0 / 25
{'eval_time_gap': 32, 'batch_size': 32, 'gamma': 0.98, 'learning_rate': 0.03372, 'target_step': 86175, 'net_dimension': 1026, 'episode': 3, 'seed': 42069}
======Model training from:  2010-01-01 to  2016-01-04
| Remove cwd: ./trained_models
Using device:  cpu
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]C:\Users\kaanb\Desktop\Code\Finance\Bot\FundSimulator\agents\agents\ppo.py:299: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  ten_reward = torch.as_tensor(temp[1], dtype=torch.float32) * reward_scale


0  1.51e+03    3.26 |    3.26    0.1    125     0 |   -0.05   0.28   0.20  -0.49


 33%|████████████████████████████                                                        | 1/3 [00:49<01:38, 49.33s/it]

0  3.02e+03    3.26 |    2.04    0.0    125     0 |    0.00   0.07   0.05  -0.49


...

0  2.11e+04    0.86 |    0.82    0.0    125     0 |   -0.01   0.60   0.32  -0.49


 89%|██████████████████████████████████████████████████████████████████████████▋         | 8/9 [04:56<00:36, 36.04s/it]

0  2.38e+04    0.86 |    0.83    0.0    125     0 |   -0.01   0.55   0.31  -0.49


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [05:32<00:00, 36.96s/it]


| UsedTime:     333 | SavedDir: ./trained_models | Total Step: 2636
======jim-ppo-recurrent-317-agent-24 Validation from:  2020-07-07 to  2021-01-04
Max reward at validation for Reccurent PPO 125.0
Model updated:  354738  times
======Trading from:  2021-01-05 to  2021-07-07
Using device:  cpu
Saving to  results/account_value_trade_main_jim-ppo-recurrent-317-agent-24.csv
previous_total_asset:2016777.7463477475
end_total_asset:2439997.2164522815
total_asset_change:423219.47010453395
Total cash is: 4.627421062186841$ and total holdings in stocks are 2439992.5890312195$
Buy & Hold strategy with previous total asset:  2324451.5253419997
Total Cost:  1219.5280758552558
Sum of rewards  382480.2158091848
Total trades:  312
Total buy orders are 207 and total sell orders are 105
Total days in turbulance:  0
Sharpe:  0.2945869661982768
Test Finished!
episode_return % 1.44
Calculating buy and hold
Buy and Hold value is  2314454.957596514
Writing hparams:  {'eval_time_gap': 256, 'batch_size': 64, '

In [11]:
0  1.51e+03   -1.38 |   -1.38    1.6    125     0 |   -0.04   0.01   0.05  -0.51
  4%|██▊                                                                             | 1/28 [04:24<1:59:10, 264.82s/it]
0  3.02e+03    3.89 |    3.89    0.1    125     0 |   -0.01   0.00  -0.05  -0.51
  7%|█████▋                                                                          | 2/28 [08:45<1:53:38, 262.26s/it]
0  4.53e+03    3.89 |    1.62    0.0    125     0 |    0.00   0.00  -0.37  -0.51
 11%|████████▌                                                                       | 3/28 [11:46<1:33:46, 225.08s/it]
0  6.03e+03    3.89 |    1.20    0.3    125     0 |   -0.01   0.00   0.21  -0.51
 14%|███████████▍                                                                    | 4/28 [14:53<1:24:04, 210.20s/it]
0  7.54e+03    3.89 |    3.62    0.1    125     0 |    0.00   0.01   0.34  -0.51
 18%|██████████████▎                                                                 | 5/28 [17:56<1:16:47, 200.31s/it]
0  9.04e+03    3.89 |    2.40    0.0    125     0 |   -0.00   0.01   0.21  -0.51
 21%|█████████████████▏                                                              | 6/28 [21:05<1:12:03, 196.53s/it]
0  1.05e+04    3.89 |    1.96    0.0    125     0 |   -0.00   0.00   0.30  -0.51
 25%|████████████████████                                                            | 7/28 [24:17<1:08:18, 195.15s/it]
0  1.21e+04    3.89 |    0.28    0.0    125     0 |   -0.00   0.00   0.10  -0.51
 29%|██████████████████████▊                                                         | 8/28 [27:24<1:04:10, 192.53s/it]
0  1.36e+04    3.89 |    2.68    0.0    125     0 |   -0.00   0.00   0.26  -0.51

0.4

In [ ]:
last = {'eval_time_gap': 8, 'batch_size': 8, 'gamma': 0.98, 'learning_rate': 0.04085,
        'target_step': 58902, 'net_dimension': 1026, 'episode': 11, 'seed': 31}